## Scalability Testing

This notebook is for testing purposes.

In [1]:
from config import DATA_PATH, Color
from helpers import clean
from helpers.coordinates import ENUPose, GRAPose
from planner import Plan
from planner.plans.auto import AutoPlan
from simulator import NoVisualizer, Simulator
from simulator.sim import SimVehicle

clean()

## Simulation Positions

In [2]:
gra_origin = GRAPose(lat=-35.3633280, lon=149.1652241,alt=0,heading=0) 
enu_origin = ENUPose(x=0, y=0, z=gra_origin.alt, heading=gra_origin.heading) 

gcs_colors=[Color.RED,Color.ORANGE,Color.GREEN,Color.BLUE]
n_uavs_per_gcs = 60
side_len = 10
altitude = 5

base_homes = ENUPose.list([
    (i * 50, j * 3 * side_len, 0, 0)
    for i in range(len(gcs_colors))
    for j in range(n_uavs_per_gcs)
])

base_paths = [
    Plan.create_square_path(side_len=side_len, alt=altitude) 
    for _ in base_homes
    ]

## Create Vehicles

In [3]:
sysids = range(1, len(base_homes)+1)
colors: list[Color]= []
for color in gcs_colors:
    colors.extend([color]*n_uavs_per_gcs)

vehs:list[SimVehicle] = []
for sysid, base_home, base_path, color in zip(sysids, base_homes, base_paths, colors):
    mission_path = DATA_PATH / f"mission_{sysid}.waypoints"
    plan = AutoPlan(
        name="simple_auto_plan",
        mission_path=str(mission_path),
    )
    plan.save_basic_mission_from_relative(
        sysid=sysid,
        gra_origin=gra_origin,
        relative_home=base_home,
        relative_path=base_path,
    )

    veh = SimVehicle.from_relative(
        sysid=sysid,
        gcs_name=f'{color.name}_{color.emoji}',
        plan=plan,
        color=color,
        enu_origin=enu_origin,
        relative_home=base_home,
        relative_path=base_path,
        model="iris",
    )
    vehs.append(veh)
    

## Simulator

In [4]:
simulator = Simulator(
	visualizer=NoVisualizer(gra_origin),
	terminals=['gcs'],
	verbose=1,
)
for veh in vehs:
    simulator.add_vehicle(veh)

## Run

In [5]:
orac = simulator.launch()
orac.run()

15:43:31 - Oracle ⚪ - INFO - 🙈 Running without visualization.
15:43:31 - Oracle ⚪ - INFO - 🚀 GCS RED_🟥 launched (PID 1233474)
15:43:31 - Oracle ⚪ - INFO - 🚀 GCS ORANGE_🟧 launched (PID 1233475)
15:43:31 - Oracle ⚪ - INFO - 🚀 GCS GREEN_🟩 launched (PID 1233476)
15:43:31 - Oracle ⚪ - INFO - 🚀 GCS BLUE_🟦 launched (PID 1233477)
15:43:31 - Oracle ⚪ - INFO - 🏁 Starting Oracle with 240 vehicles and 4 GCSs
15:45:24 - Oracle ⚪ - INFO - UAV 124 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 70 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 152 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 130 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 28 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 63 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 206 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 140 completed mission and exited
15:45:25 - Oracle ⚪ - INFO - UAV 141 completed mission and exited
15:45:25 